In [27]:
import pandas as pd
from scripts.fetch_comments import fetch_comments, save_to_csv

In [28]:

video_id = "JfVOs4VSpmA"
comments = fetch_comments(video_id,1000)
save_to_csv(comments, "./data/comments.csv")
print(f"Total comments fetched: {len(comments)}")

Saved 100 comments to ./data/comments.csv
Total comments fetched: 100
